# Static Templates Use Here for sent the mail

In [2]:
import time
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.errors import HttpError
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import base64

# Constants
LOG_FILE = "Email Logging.xlsx"
SCOPES = ['https://www.googleapis.com/auth/gmail.send', 'https://www.googleapis.com/auth/gmail.readonly']
EMAIL_TEMPLATES = [
    {"subject": "Collaboration Opportunity: ABC and {company_name}", "body": "Dear {company_name}, First email content..."},
    {"subject": "Following Up: Excited to Connect with {company_name}", "body": "Dear {company_name}, Second email content..."},
    {"subject": "Wrapping Up: Collaboration Between ABC and {company_name}", "body": "Dear {company_name}, Third email content..."}
]
TIME_INTERVALS = [30, 90, 120]  # Time intervals in seconds

# Step 1: Gmail Authentication
def authenticate_gmail():
    """Authenticate and initialize Gmail API."""
    try:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=6231)
        return build('gmail', 'v1', credentials=creds)
    except Exception as e:
        print(f"Error occurred during Gmail authentication: {e}")
        raise

# Step 2: Send Email to a Single Recipient
def send_email(service, email_id, subject, body):
    """Send an email to a single recipient using Gmail API."""
    try:
        message = MIMEMultipart()
        message['to'] = email_id
        message['subject'] = subject
        message.attach(MIMEText(body))
        raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
        sent_message = service.users().messages().send(userId="me", body={'raw': raw_message}).execute()
        return sent_message['id']  # Return the message ID to track the thread
    except HttpError as error:
        print(f"HttpError occurred while sending the email: {error}")
        raise
    except Exception as e:
        print(f"Unexpected error occurred while sending the email: {e}")
        raise

# Step 3: Check for Replies
def check_for_replies(service, sent_message_id, recipient_email):
    """Check for replies to the sent message."""
    if not sent_message_id:
        print("No message ID found to track replies.")
        return False

    try:
        # Get the thread ID of the sent email
        sent_message = service.users().messages().get(userId="me", id=sent_message_id).execute()
        thread_id = sent_message.get('threadId')

        if not thread_id:
            print(f"No thread ID found for the sent message with ID {sent_message_id}")
            return False

        # Search for all messages in the thread
        messages = service.users().threads().get(userId='me', id=thread_id).execute().get('messages', [])
        for message in messages:
            # Get the metadata of each message
            headers = {header['name']: header['value'] for header in message['payload']['headers']}
            sender = headers.get('From', '')

            # Check if the sender is the recipient of the original email
            if recipient_email in sender:
                print(f"Reply detected from {recipient_email}.")
                return True

        print("No reply detected in the thread.")
        return False
    except HttpError as error:
        print(f"HttpError occurred while checking for replies: {error}")
        return False
    except Exception as e:
        print(f"Unexpected error occurred while checking for replies: {e}")
        return False

# Step 4: Update Logs
def update_logs(email_id, status):
    """Update the log file."""
    try:
        df = pd.read_excel(LOG_FILE)
    except FileNotFoundError:
        df = pd.DataFrame(columns=["Email ID", "Status", "Timestamp"])
    except Exception as e:
        print(f"Error occurred while reading the log file: {e}")
        raise

    new_entry = {"Email ID": email_id, "Status": status, "Timestamp": pd.Timestamp.now()}
    try:
        df = pd.concat([df, pd.DataFrame([new_entry])], ignore_index=True)
        df.to_excel(LOG_FILE, index=False)
    except Exception as e:
        print(f"Error occurred while updating the log file: {e}")
        raise

# Step 5: Email Pipeline for Multiple Recipients and Different Company Names
def email_pipeline(service, recipients):
    """Pipeline to send periodic emails to different recipients with different company names and check for replies."""
    for recipient in recipients:
        email_id = recipient['email_id']
        company_name = recipient['company_name']

        print(f"Starting email pipeline for {email_id} - {company_name}...")
        
        sent_message_id = None
        for i, template in enumerate(EMAIL_TEMPLATES):
            try:
                subject = template["subject"].format(company_name=company_name)
                body = template["body"].format(company_name=company_name)

                # Send the email
                sent_message_id = send_email(service, email_id, subject, body)
                if not sent_message_id:
                    print(f"Error: Could not send email to {email_id}.")
                    update_logs(email_id, "Email Sending Failed")
                    break

                print(f"Email {i+1} sent to {email_id}")
                update_logs(email_id, f"Email {i+1} Sent")

                # Wait for a reply to the current email
                print(f"Waiting for {TIME_INTERVALS[i]} seconds...")
                for _ in range(TIME_INTERVALS[i] // 5):  # Check every 5 seconds
                    time.sleep(5)
                    if check_for_replies(service, sent_message_id, email_id):
                        print(f"Reply received from {email_id}. Terminating pipeline.")
                        update_logs(email_id, "Reply Received")
                        break
                else:
                    print(f"No reply after email {i+1}. Moving to next step.")

                # If the reply is received, break the loop to stop sending further emails
                if sent_message_id and check_for_replies(service, sent_message_id, email_id):
                    break
            except Exception as e:
                print(f"Error occurred during email pipeline for {email_id} - {company_name}: {e}")
                update_logs(email_id, f"Error: {e}")
                break
        else:
            # If no reply after all emails, mark the end
            print(f"No reply received after all emails to {email_id}. Pipeline completed.")
            update_logs(email_id, "No Reply Received")

# Main Execution
if __name__ == "__main__":
    try:
        service = authenticate_gmail()

        # List of recipients with different email IDs and company names
        recipients = [
            {"email_id": "rajagurjar83@gmail.com", "company_name": "Gurjar Raja"},
            {"email_id": "ravi.tiwari9603@gmail.com", "company_name": "Rasu"}
        ]

        email_pipeline(service, recipients)
    except Exception as e:
        print(f"Error occurred during main execution: {e}")


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=731769731106-j14kqkaav4d7p5hlh15qis7mpea9sofc.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A6231%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=J0IffrnCElrDmgOkfF5XHyGrwexXm5&access_type=offline
Starting email pipeline for rajagurjar83@gmail.com - Gurjar Raja...
Email 1 sent to rajagurjar83@gmail.com
Waiting for 30 seconds...
No reply detected in the thread.
No reply detected in the thread.
No reply detected in the thread.
No reply detected in the thread.
No reply detected in the thread.
Reply detected from rajagurjar83@gmail.com.
Reply received from rajagurjar83@gmail.com. Terminating pipeline.
Reply detected from rajagurjar83@gmail.com.
Starting email pipeline for ravi.tiwari9603@gmail.com - Rasu...
Email 1 sent to ravi.tiwari9603@gmail.com
Waiting for 30 seconds...


# Main Code of Email Agent (Read the docx file for email templates)

In [2]:
import time
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.errors import HttpError
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import base64
from docx import Document

# Constants
LOG_FILE = "Email Logging.xlsx"
SCOPES = ['https://www.googleapis.com/auth/gmail.send', 'https://www.googleapis.com/auth/gmail.readonly']
TIME_INTERVALS = [30, 90, 120]  # Time intervals in seconds

# Step 1: Load Email Templates from Word Document
def load_email_templates(docx_path):
    """Load email templates from a Word document with structured format."""
    try:
        doc = Document(docx_path)
        templates = []
        current_template = {}

        for para in doc.paragraphs:
            text = para.text.strip()  # Clean up the paragraph text
            if not text:  # Skip empty lines
                continue

            # Detect and handle Email_subject
            if text.lower().startswith("email_subject:"):
                if current_template:  # Save the previous template if it exists
                    templates.append(current_template)
                current_template = {"subject": text.split(":", 1)[1].strip(), "body": ""}
            
            # Detect and handle Email_body
            elif text.lower().startswith("email_body:"):
                current_template["body"] = ""
            
            # Add lines to the email body
            else:
                if "body" in current_template:
                    current_template["body"] += f"{text}\n"

        # Append the last template if exists
        if current_template:
            templates.append(current_template)

        return templates
    except Exception as e:
        print(f"Error occurred while reading Word document: {e}")
        raise

# Step 2: Gmail Authentication
def authenticate_gmail():
    """Authenticate and initialize Gmail API."""
    try:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=6231)
        return build('gmail', 'v1', credentials=creds)
    except Exception as e:
        print(f"Error occurred during Gmail authentication: {e}")
        raise

# Step 3: Send Email to a Single Recipient
def send_email(service, email_id, subject, body):
    """Send an email to a single recipient using Gmail API."""
    try:
        message = MIMEMultipart()
        message['to'] = email_id
        message['subject'] = subject
        message.attach(MIMEText(body))
        raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
        sent_message = service.users().messages().send(userId="me", body={'raw': raw_message}).execute()
        return sent_message['id']  # Return the message ID to track the thread
    except HttpError as error:
        print(f"HttpError occurred while sending the email: {error}")
        return None
    except Exception as e:
        print(f"Unexpected error occurred while sending the email: {e}")
        raise

# Step 4: Check for Replies
def check_for_replies(service, sent_message_id, recipient_email):
    """Check for replies to the sent message."""
    if not sent_message_id:
        print("No message ID found to track replies.")
        return False

    try:
        # Get the thread ID of the sent email
        sent_message = service.users().messages().get(userId="me", id=sent_message_id).execute()
        thread_id = sent_message.get('threadId')

        if not thread_id:
            print(f"No thread ID found for the sent message with ID {sent_message_id}")
            return False

        # Search for all messages in the thread
        messages = service.users().threads().get(userId='me', id=thread_id).execute().get('messages', [])
        for message in messages:
            # Get the metadata of each message
            headers = {header['name']: header['value'] for header in message['payload']['headers']}
            sender = headers.get('From', '')

            # Check if the sender is the recipient of the original email
            if recipient_email in sender:
                print(f"Reply detected from {recipient_email}.")
                return True

        print("No reply detected in the thread.")
        return False
    except HttpError as error:
        print(f"HttpError occurred while checking for replies: {error}")
        return False
    except Exception as e:
        print(f"Unexpected error occurred while checking for replies: {e}")
        return False

# Step 5: Update Logs
def update_logs(email_id, status):
    """Update the log file."""
    try:
        df = pd.read_excel(LOG_FILE)
    except FileNotFoundError:
        df = pd.DataFrame(columns=["Email ID", "Status", "Timestamp"])
    except Exception as e:
        print(f"Error occurred while reading the log file: {e}")
        raise

    new_entry = {"Email ID": email_id, "Status": status, "Timestamp": pd.Timestamp.now()}
    try:
        df = pd.concat([df, pd.DataFrame([new_entry])], ignore_index=True)
        df.to_excel(LOG_FILE, index=False)
    except Exception as e:
        print(f"Error occurred while updating the log file: {e}")
        raise

# Step 6: Email Pipeline for Multiple Recipients and Different Company Names
def email_pipeline(service, recipients, email_templates):
    """Pipeline to send periodic emails to different recipients with different company names and check for replies."""
    for recipient in recipients:
        email_id = recipient['email_id']
        company_name = recipient['company_name']

        print(f"Starting email pipeline for {email_id} - {company_name}...")
        
        sent_message_id = None
        for i, template in enumerate(email_templates):
            try:
                subject = template["subject"].format(company_name=company_name)
                body = template["body"].format(company_name=company_name)

                # Send the email
                sent_message_id = send_email(service, email_id, subject, body)
                if not sent_message_id:
                    print(f"Error: Could not send email to {email_id}.")
                    update_logs(email_id, "Email Sending Failed")
                    break

                print(f"Email {i+1} sent to {email_id}")
                update_logs(email_id, f"Email {i+1} Sent")

                # Wait for a reply to the current email
                print(f"Waiting for {TIME_INTERVALS[i]} seconds...")
                for _ in range(TIME_INTERVALS[i] // 5):  # Check every 5 seconds
                    time.sleep(5)
                    if check_for_replies(service, sent_message_id, email_id):
                        print(f"Reply received from {email_id}. Terminating pipeline.")
                        update_logs(email_id, "Reply Received")
                        break
                else:
                    print(f"No reply after email {i+1}. Moving to next step.")

                # If the reply is received, break the loop to stop sending further emails
                if sent_message_id and check_for_replies(service, sent_message_id, email_id):
                    break
            except Exception as e:
                print(f"Error occurred during email pipeline for {email_id} - {company_name}: {e}")
                update_logs(email_id, f"Error: {e}")
                break
        else:
            # If no reply after all emails, mark the end
            print(f"No reply received after all emails to {email_id}. Pipeline completed.")
            update_logs(email_id, "No Reply Received")

# Main Execution
if __name__ == "__main__":
    try:
        # Load email templates from the Word file
        email_templates = load_email_templates("Email Templates Sales agent.docx")

        service = authenticate_gmail()

        # List of recipients with different email IDs and company names
        recipients = [
            {"email_id": "rajagurjar83@gmail.com", "company_name": "Gurjar Raja"},
            {"email_id": "ravi.tiwari9603@gmail.com", "company_name": "Rasu"}
        ]

        email_pipeline(service, recipients, email_templates)
    except Exception as e:
        print(f"Error occurred during main execution: {e}")

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=731769731106-j14kqkaav4d7p5hlh15qis7mpea9sofc.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A6231%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=41qr2dFPjGhNefaRJyC2xuVd6B9DGn&access_type=offline
Starting email pipeline for rajagurjar83@gmail.com - Gurjar Raja...
Email 1 sent to rajagurjar83@gmail.com
Waiting for 30 seconds...
No reply detected in the thread.
No reply detected in the thread.
No reply detected in the thread.
No reply detected in the thread.
No reply detected in the thread.
Reply detected from rajagurjar83@gmail.com.
Reply received from rajagurjar83@gmail.com. Terminating pipeline.
Reply detected from rajagurjar83@gmail.com.
Starting email pipeline for ravi.tiwari9603@gmail.com - Rasu...
Email 1 sent to ravi.tiwari9603@gmail.com
Waiting for 30 seconds...


In [22]:
from docx import Document

def load_email_templates(docx_path):
    """Load email templates from a Word document with structured format."""
    try:
        doc = Document(docx_path)
        templates = []
        current_template = {}

        for para in doc.paragraphs:
            text = para.text.strip()  # Clean up the paragraph text
            if not text:  # Skip empty lines
                continue

            # Detect and handle Email_subject
            if text.startswith("Email_subject:"):
                if current_template:  # Save the previous template if it exists
                    templates.append(current_template)
                current_template = {"subject": text.replace("Email_subject:", "").strip(), "body": ""}
            
            # Detect and handle Email_body
            elif text.startswith("Email_body:"):
                current_template["body"] = ""
            
            # Add lines to the email body
            else:
                if "body" in current_template:
                    current_template["body"] += f"{text}\n"

        # Append the last template if exists
        if current_template:
            templates.append(current_template)

        return templates
    except Exception as e:
        print(f"Error while reading Word document: {e}")
        raise

# Example Usage
docx_path = "Email Templates Sales agent.docx"
email_templates = load_email_templates(docx_path)

for i, template in enumerate(email_templates):
    print(f"Sending: {template['subject']}")
    print(f"Sending: {template['body']}")
    


Sending: Collaboration Opportunity: ABC and {company_name}
Sending: I hope this message finds you well! I'm John, Sales manager at ABC. I came across {company_name} and was impressed by your work in the AI/ML and Software development space.
At ABC, we specialize in AI Applications and Full-Stack Development. Given {company_name}'s focus on AI/ML and Software development, I believe there's potential for a meaningful partnership between our organizations.
Would it be possible to schedule a quick call to explore how ABC and {company_name} might work together? We're confident we can create value for both sides.
Looking forward to hearing from you!
Best regards,
John Doe
Sales manager
john@abc.com
+1-8765-434-568

Sending: Following Up: Excited to Connect with {company_name}
Sending: I hope you had a chance to review my earlier message. I wanted to follow up to see if {company_name} might be interested in discussing a potential collaboration with ABC.
At ABC, we're passionate about AI advan